# Search-tweets-python

In [ ]:
import json
from searchtweets import ResultStream, gen_rule_payload, load_credentials

# Cargar credenciales
premium_search_args = load_credentials("./twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

# Lista de usuarios
usuarios = [
    'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
    'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
    'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
    'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
    'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
    'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
    'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
    'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
    'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
    'CiudaDanaMtySur'
]

# Crear la regla de búsqueda con filtro de fechas para el año 2024
rule = gen_rule_payload(
    " OR ".join([f"from:{user}" for user in usuarios]),  # Se arma la query con cada usuario
    from_date="2024-01-01", 
    to_date="2024-12-31",
    results_per_call=50
)

# Ejecutar la búsqueda
rs = ResultStream(rule_payload=rule, max_results=50, **premium_search_args)

# Obtener resultados
tweets = list(rs.stream())

# Guardar los resultados en un archivo JSON
with open('tweets_2024.json', 'w', encoding='utf-8') as f:
    json.dump(tweets, f, ensure_ascii=False, indent=4)

print("Tweets de 2024 guardados en 'tweets_2024.json'")

## Intento con Requests

In [2]:
import tweepy
import time
import csv
from datetime import datetime
import pytz
import os

# Claves de autenticación
BEARER_TOKEN = os.getenv("BEARER_TOKEN")

# Autenticación mediante Bearer Token
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Lista de usuarios por sus nombres de usuario
usuarios = [
    'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
    'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
    'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
    'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
    'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
    'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
    'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
    'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
    'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
    'CiudaDanaMtySur']

# Función para obtener los tweets de un usuario limitados a 2024
def get_user_tweets(username, count):
    try:
        # Obtener el ID del usuario
        user = client.get_user(username=username)
        user_id = user.data.id
        
        # Limitar los tweets a partir de enero de 2024
        start_time = datetime(2024, 1, 1).isoformat() + 'Z'
        
        # Obtener los tweets del usuario con los campos necesarios
        tweets = client.get_users_tweets(
            id=user_id,
            max_results=count,
            start_time=start_time,
            tweet_fields=['created_at', 'public_metrics', 'entities', 'geo']
        )
        
        # Verificar si se obtuvieron tweets correctamente
        if tweets.data:
            print(f"Tweets obtenidos correctamente para el usuario {username}")
            return tweets.data
        else:
            print(f"No se encontraron tweets para el usuario {username}")
            return None

    except tweepy.TweepyException as e:
        print(f"Error al obtener tweets para el usuario {username}: {e}")
        return None

# Manejo de Rate Limits y revisión de éxito
def get_tweets_multiple_users(usuarios, count):
    # Definir la zona horaria de Monterrey
    zona_horaria_monterrey = pytz.timezone('America/Monterrey')
    
    tweets_por_usuario = {}
    for usuario in usuarios:
        try:
            tweets = get_user_tweets(usuario, count)
            if tweets:
                # Guardar 'created_at', 'public_metrics', 'entities' y 'geo' de cada tweet
                tweets_por_usuario[usuario] = [
                    {
                        'username': usuario,
                        'created_at': tweet.created_at,
                        'public_metrics': tweet.public_metrics,
                        'entities': tweet.entities,
                        'geo': tweet.geo
                    }
                    for tweet in tweets
                ]
                print(f"Datos procesados correctamente para el usuario {usuario}")
            else:
                print(f"Sin datos para procesar para el usuario {usuario}")

        except tweepy.TooManyRequests:
            # Obtener la hora actual en la zona horaria de Monterrey cuando se detecta el rate limit
            hora_actual_monterrey = datetime.now(zona_horaria_monterrey).strftime("%Y-%m-%d %H:%M:%S")
            print(f"Rate limit alcanzado a las {hora_actual_monterrey}. Esperando 15 minutos...")
            time.sleep(900)  # Pausa de 15 minutos para respetar el rate limit

        except Exception as e:
            print(f"Error inesperado para el usuario {usuario}: {e}")
    
    return tweets_por_usuario

# Obtener los tweets de los usuarios
tweets_por_usuario = get_tweets_multiple_users(usuarios, 50)

# Guardar los resultados en un archivo CSV
with open('tweets_usuarios.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['username', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'entities', 'geo']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for usuario, tweets in tweets_por_usuario.items():
        for tweet in tweets:
            writer.writerow({
                'username': usuario,
                'created_at': tweet['created_at'],
                'retweet_count': tweet['public_metrics']['retweet_count'],
                'reply_count': tweet['public_metrics']['reply_count'],
                'like_count': tweet['public_metrics']['like_count'],
                'quote_count': tweet['public_metrics']['quote_count'],
                'entities': tweet['entities'],
                'geo': tweet['geo']
            })

print("Tweets guardados en 'tweets_usuarios.csv'")

Error al obtener tweets para el usuario Cicmty: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario Cicmty
Error al obtener tweets para el usuario conagua_mx: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario conagua_mx
Error al obtener tweets para el usuario elnortelocal: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario elnortelocal
Error al obtener tweets para el usuario LoQuePasaenNL: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario LoQuePasaenNL
Error al obtener tweets para el usuario eldelaconsty: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario eldelaconsty
Error al obtener tweets para el usuario camion_desde: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario camion_desde
Error al obtener tweets para el usuario metrorreynlofi1: 401 Unauthorized
Unauthorized
Sin datos para procesar para el usuario metrorreynlofi1
Error al obtener tweets para el usuario Movilidad

## Intento con twitter_scrapper v 2020

In [2]:
from twitter_scraper import get_tweets
import json

# Lista de usuarios
usuarios = [
    'Cicmty', 'conagua_mx', 'elnortelocal', 'LoQuePasaenNL', 'eldelaconsty', 'camion_desde',
    'metrorreynlofi1', 'MovilidadSN', 'SSP_Apodaca', 'PC_NuevoLeon', 'TodosSomosHLM',
    'AtentosMTYSur', 'Apodaca_News', 'ArbSanJorge', 'SoyDeSanNicolas', 'abimaelsal',
    'BomberosNL', 'nelvaldez', 'MAGS_SP', 'saz2000', 'DarkKnightMty', 'revistacodigo21',
    'Mty_Leones', 'cesarmty', '911SanPedro', 'SSPCMonterrey', 'pc_mty', 'rayelizalder',
    'Informativo3651', 'arualsanpedrogg', 'JulioCesarCano', 'nmasmonterrey', 'info7mty',
    'ABCNoticiasMX', 'AsiEsMonterrey', 'GobSanNicolas', 'PortalElPunto',
    'gob_Escobed', 'mtygob', 'joluisgarcia', 'visionmty1', 'QuePasaEnS', 'LCAlatorre',
    'IdentidadNL', 'MtyFollow', 'ayd_monterrey', 'ComunidadMTY', 'municipiodegpe',
    'CiudaDanaMtySur'
]

# Obtener los tweets de los usuarios
tweets_por_usuario = {}

for usuario in usuarios:
    try:
        # Obtener los tweets del usuario
        tweets = get_tweets(usuario, pages=1)
        
        # Guardar 'time' y 'text' de cada tweet
        tweets_por_usuario[usuario] = [
            {
                'time': tweet['time'],
                'text': tweet['text']
            }
            for tweet in tweets
        ]
        print(f"Tweets obtenidos correctamente para el usuario {usuario}")

    except Exception as e:
        print(f"Error al obtener tweets para el usuario {usuario}: {e}")

# Guardar los resultados en un archivo JSON
with open('tweets_usuarios.json', 'w', encoding='utf-8') as f:
    json.dump(tweets_por_usuario, f, ensure_ascii=False, indent=4)

print("Tweets guardados en 'tweets_usuarios.json'")

Error al obtener tweets para el usuario Cicmty: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario conagua_mx: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario elnortelocal: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario LoQuePasaenNL: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario eldelaconsty: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario camion_desde: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario metrorreynlofi1: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario MovilidadSN: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario SSP_Apodaca: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario PC_NuevoLeon: Expecting value: line 1 column 1 (char 0)
Error al obtener tweets para el usuario TodosS

## Intento con ScrapFly

In [3]:
!pip install playwright jmespath scrapfly-sdk

   ---------------------------------------- 0.0/29.9 MB ? eta -:--:--
   -- ------------------------------------- 2.1/29.9 MB 14.7 MB/s eta 0:00:02
   -------- ------------------------------- 6.3/29.9 MB 17.5 MB/s eta 0:00:02
   ------------- -------------------------- 10.0/29.9 MB 17.2 MB/s eta 0:00:02
   ------------------ --------------------- 13.6/29.9 MB 17.5 MB/s eta 0:00:01
   --------------------- ------------------ 16.3/29.9 MB 16.2 MB/s eta 0:00:01
   ------------------------- -------------- 19.4/29.9 MB 16.1 MB/s eta 0:00:01
   ----------------------------- ---------- 22.3/29.9 MB 15.8 MB/s eta 0:00:01
   ---------------------------------- ----- 25.4/29.9 MB 15.8 MB/s eta 0:00:01
   -------------------------------------- - 28.6/29.9 MB 15.6 MB/s eta 0:00:01
   ---------------------------------------- 29.9/29.9 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pyee
    Found existing installation: pyee 11.1.1
    Uninstalling pyee-11.1.1:
      Successfully uninstalled pyee-11

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires pyee<12.0.0,>=11.0.0, but you have pyee 12.0.0 which is incompatible.


## Intento con twscrape

In [ ]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level

